In [7]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from google.cloud import bigquery
from bq_carrefour import MethodBQ
import numpy as np

In [ ]:
'''
Funcion para normalizar el padron y dejarlo Limpio y operativo
'''
### COMIENZO A TRABAJAR SOBRE EL PADRON ###
    # Cargo la Informacion del padron
try:
    # Indico que columnas voy a necesitar
    cols = ['GSX', 'NOMBRE', 'Fecha apertura', 'FIN DE CIERRE','ORGANIZACIÓN ', 'DIRECTOR EXPLOTACIÓN', 'DIRECTOR OPERACIONAL', 'DIRECTOR / GERENTE REGIONAL', 'SUB REGION', 'DIRECTOR/ GERENTE TIENDA', 'Provincia Tableau', 'M² SALÓN', 'M² PGC', 'M² PFT', 'M² BAZAR', 'M² Electro', 'M² Textil', 'M² Pls', 'M² GALERIAS', 'PROVINCIA', 'M² Parcking', 'CAJAS', 'COD.POSTAL']
    pad = pd.read_excel('data/padron.xlsx', header=17, usecols=cols)

except Exception as e:
    print('Error a la hora de cargar el Padron. Detalle {e}')

# Estandarizo un poco los nombres de las columnas
pad.columns = pad.columns.str.strip().str.lower().str.replace(' /', '').str.replace('/', '').str.replace('.', ' ')

# Quito nulos
pad = pad.dropna(subset=['nombre', 'organización', 'fecha apertura'])

# Me quedo unicamente con formatos validos
pad = pad[pad['organización'].isin(['HIPERMERCADO', 'MAXI', 'Market', 'Express', 'E-Commerce'])]

# Genero una columna de Estado
pad['estado'] = np.where(pad['fin de cierre'] == '-', 'Activa', 'Cerrada')

# Quito columna incompleta
pad = pad.drop(columns={'provincia tableau'})

# Renombro la columna completa de provincias
pad = pad.rename(columns={
    'provincia':'provincia tableau',
    'gsx':'id tienda'
})

# Genero un list aux para bucle abajo
cols = ['m² pgc', 'm² pft', 'm² bazar', 'm² electro', 'm² textil', 'm² pls', 'm² galerias', 'cajas', 'm² parcking']

# Itero sobre cols para estandarizar, rellenar y limpiar nulos
for col in cols:
    pad[col] = pad[col].fillna(0).replace('-', 0).replace('sd', 0).replace('SD', 0).replace('', 0).astype(int)

# Ordeno columnas
pad = pad[['nombre', 'id tienda', 'fecha apertura', 'organización', 'director explotación', 'director operacional', 'director gerente regional', 'sub region', 'director gerente tienda', 'provincia tableau', 'm² salón', 'm² pgc', 'm² pft', 'm² bazar', 'm² electro', 'm² textil', 'm² pls', 'm² galerias', 'cod postal', 'cajas', 'm² parcking', 'estado']]

# Convierto el codigo postal a String ya que es alfanumerico
pad['cod postal'] = pad['cod postal'].astype(str)

pad['fecha apertura'] = pd.to_datetime(pad['fecha apertura'], format='%Y/%m/%d', errors='coerce')

# Genero una nueva columna para identificar la ultima fecha de actualizacion de las tiendas
pad['modificacion'] = datetime.today().strftime('%d/%m/%Y')

# Formateo la fecha y la convierto a String. Como no es util para comprar o utilizar en series de tiempo, sirve igual
pad['modificacion'] = pad['modificacion'].astype('str')

# Reseteo y quito Indice indeseado
pad = pad.reset_index(drop=True)

# Realizo una transformacion para Normalizar valores de las provincias
pad['provincia tableau'] = pad['provincia tableau'].str.strip().str.upper()
pad['provincia tableau'] = np.where(pad['provincia tableau'] == 'NEUQUÉN', 'NEUQUEN', pad['provincia tableau'])

#Remplazo cualquier guion por nans
pad = pad.replace('-', np.nan)

In [84]:
def crear_tabla_si_no_existe(df: pd.DataFrame, table_id: str, project_id: str):
    client = bigquery.Client(project=project_id)

    try:
        client.get_table(table_id)
        print(f"✅ La tabla {table_id} ya existe")
    except NotFound:
        print(f"⚠️ La tabla {table_id} no existe. Creándola...")

        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_EMPTY"
        )

        load_job = client.load_table_from_dataframe(
            dataframe=df,
            destination=table_id,
            job_config=job_config
        )

        load_job.result()
        print(f"✅ Tabla {table_id} creada correctamente.")

In [85]:
def carga_padron(padron, project_id='gcp-ar-cdg-datos-dev', dataset_id='marketshare_project', table_name='padron'):
    try:
        client = bigquery.Client(project=project_id)

        # Construimos correctamente el ID completo de la tabla
        full_table_id = f"{project_id}.{dataset_id}.{table_name}"

        # Paso previo: crear tabla si no existe
        crear_tabla_si_no_existe(padron, full_table_id, project_id)

        # Configuración del job de carga
        job_config = bigquery.LoadJobConfig(
            autodetect=True,
            write_disposition="WRITE_TRUNCATE"  # 👈 Sobrescribir información
        )
        
        # Subir el DataFrame
        job = client.load_table_from_dataframe(padron, full_table_id, job_config=job_config)
        job.result()

        print(f"✅ Éxito al cargar la información del padrón a GCP: {full_table_id}")

    except Exception as e:
        print(f"❌ Error al subir el padrón a GCP: {e}")

In [13]:
import pandas as pd

df = pd.read_excel('data/padron.xlsx', header=17)

In [14]:
df.columns.to_list()

['Unnamed: 0',
 'GSX',
 'BW',
 'BFC entity',
 'BFC format',
 'SAP',
 'TD',
 'Pegasus',
 'PREV NAMES',
 'PREV ALIAS',
 'NOMBRE',
 'Fecha apertura',
 'BANDERA',
 'ORGANIZACIÓN ',
 'DIRECTOR EXPLOTACIÓN',
 'DIRECTOR OPERACIONAL',
 'DIRECTOR / GERENTE REGIONAL',
 'SUB REGION',
 'DIRECTOR/ GERENTE TIENDA',
 'MODELO',
 'Fecha Lanzamiento',
 'CONTROLADOR',
 'latitude',
 'longitude',
 'Provincia Tableau',
 'Provincia Shapefile',
 'Municipalidad Shapefile',
 'M² SALÓN',
 'M² PGC',
 'M² PFT',
 'M² BAZAR',
 'M² Electro',
 'M² Textil',
 'M² Pls',
 'M² GALERIAS',
 'DOMICILIO',
 'COD.POSTAL',
 'CELULAR TIENDA',
 'BARRIO/LOCALIDAD',
 'PROVINCIA',
 'MUNICIPALIDAD',
 'CELULAR RESPONSABLE',
 'PROPIEDAD',
 'PARQUE',
 'FECHA CAMBIO DE BANDERA',
 'EX BANDERA',
 'DIAS CERRADOS',
 'CAJAS',
 'M² Parcking',
 'INICIO DE REFORMA',
 'FIN DE REFORMA',
 'FIN DE CIERRE',
 'Transformaciones',
 's',
 'clase',
 'Unnamed: 55',
 'Unnamed: 56',
 'Unnamed: 57',
 'Unnamed: 58',
 'Unnamed: 59',
 'Unnamed: 60',
 'Unnamed: 61'

### ARREGLO DE MARKETSHARE

In [37]:
try:
    df = pd.read_excel('data/share_dic_25.xlsx')
except Exception as e:
    print(f'Error al cargar la informacion del Share. Detalle: {e}')

# Comienzo a trabajar sobre los cambios en el archivo del Share
# Genero un bucle para convertir los nombres de las columnas y que tengan mas sentido. Si la columna es datetime, entonces se obtiene el nombre del mes (En ingles) y se lo coloca como nuevo nombre concatenado con su año
for col in df.columns:

    if isinstance(col, datetime):
        month = col.strftime('%B')
        year = col.year
        new_name = f'{month.lower()} {year}'
        df = df.rename(columns={col:str(new_name)})

    else:
        new_name = col.lower()
        df = df.rename(columns={col:new_name})

# Trabajo sobre el numero Operacional
df['succad'] = df['succad'].replace(np.nan, 0)
df['succad'] = df['succad'].astype(int)
df['succad'] = df['succad'].replace(0, 'RESTO')

    # Renombro columnas
df = df.rename(columns=
    {
    'area1_rs':'area',
    'area_scentia_rs':'region',
    'mercado reporte':'subregion',
    'formato_m2_rs':'formato_m2',
    'mdo carrefour':'marca',
    'bandera carrefour':'formato',
    'succad':'numero_operacional'
    }
)

# Doy formato y estandarizo los rangos de superficie
df['formato_m2'] = df['formato_m2'].str.split('-', n=1).str[1].str.replace('-', 'a')

# Genero diccionario Auxiliar con los meses en ingles y castellano
meses_en = {
'january': 'enero',
'february': 'febrero',
'march': 'marzo',
'april': 'abril',
'may': 'mayo',
'june': 'junio',
'july': 'julio',
'august': 'agosto',
'september': 'septiembre',
'october': 'octubre',
'november': 'noviembre',
'december': 'diciembre'
}
# Almaceno los nuevos nombres de las columnas
nuevo_nombre_cols = {}

for col in df.columns:
    partes = col.lower().split(' ', 1)
    if partes[0] in meses_en:
        nuevo_nombre = f"{meses_en[partes[0]]} {partes[1]}"
        nuevo_nombre_cols[col] = nuevo_nombre

# Cambio los nombres viejos por los nuevos
df = df.rename(columns=nuevo_nombre_cols)

# Saco columnas innecesarias
df = df.drop(columns=['ytd24', 'ytd25'])

# PASO CRITICO / DERRITO EL DF PARA TENER EL DATO DE LAS FECHAS COMO VARIABLE CATEGORICA Y PASAR DE UN FORMATO WIDE A LONG
df = df.melt(id_vars=['area', 'region', 'subregion', 'formato_m2', 'marca', 'formato', 'numero_operacional'], var_name='fecha', value_name='ventas_con_tasa')

# Genero columnas
df['mes'] = df['fecha'].str.split(' ').str[0]
df['año'] = df['fecha'].str.split(' ').str[1]

# Ordeno el DF
df = df[['area', 'region', 'subregion', 'formato_m2', 'marca', 'formato', 'numero_operacional', 'fecha', 'mes', 'año', 'ventas_con_tasa']]

# Me aseguro que la columna Num Op sea Numero
df['numero_operacional'] = df['numero_operacional'].astype(str)

# Me quedo unicamente con los registros que tienen Venta
df = df[df['ventas_con_tasa'] != 0]

# Quito valores nulos
df = df.dropna(axis=0, subset=['ventas_con_tasa'], how='any')

# Convierto las ventas en INT
df['ventas_con_tasa'] = df['ventas_con_tasa'].astype(int)

# Genero nuevas columnas
df['ventas_con_tasa_millones'] = df['ventas_con_tasa'] / 1_000_000
df['ventas_con_tasa_millones'] = df['ventas_con_tasa_millones'].astype(float).round(2)

# Estandarizo los formatos
df['formato'] = df['formato'].replace(
    {
    'CARREFOUR EXPRESS':'EXPRESS',
    'CARREFOUR HIPER':'HIPER',
    'CARREFOUR MARKET':'MARKET',
    'CARREFOUR MAXI':'MAXI'
    }
)

# Diccionario Auxiliar para generar una columna de "fecha_parsed"
meses_a_numero = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

# Genero columnas auxiliares para generar una columna Datetime
df['numero_mes'] = df['mes'].map(meses_a_numero)
df['fecha_parsed'] = df['numero_mes'] + '/' + '01/' + df['año']
df['fecha_final'] = pd.to_datetime(df['fecha_parsed'])

# Elimino las columnas auxiliares
df = df.drop(columns=['numero_mes', 'fecha_parsed', 'fecha'])

In [ ]:
# Genero una variable que capture la fecha en la cual se ejecuta el flujo y le resto 28 dias para obtener el mes, y año correcta para filtrar la informacion y que solamente se suba la informacion del mes vencido a la tabla de GCP para no arruinar el historico.
fecha_pivot = pd.to_datetime(datetime.today().date()) - timedelta(days=28)

# Tomo la info de la fecha pivot y en cambio de quedarme con el dia, le coloco siempre a la fecha el primero de cada mes para que sea comparable con el reporte de Sentia
fecha_pivot = pd.to_datetime(str(fecha_pivot.year) + '/' + str(fecha_pivot.month)+ '/01', errors='coerce', format='%Y/%m/%d')

# Filtro la informacion y me quedo unicamente con la informacion del mes que voy a concatenar al Historico
df['fecha_final'] = pd.to_datetime(df['fecha_final'], errors='coerce', format='%Y/%m/%d')
df = df[df['fecha_final'] == fecha_pivot]